In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, GET
import pandas as pd

In [ ]:
# Set sparql endpoint

sparql_endpoint = ""

In [ ]:
# Configure sparql wrapper, default request method GET, default response format JSON

def query_sparql(sparql_endpoint, query, query_method=GET, response_format=JSON):
    sparql_wrapper = SPARQLWrapper(sparql_endpoint)
    sparql_wrapper.addCustomHttpHeader("Content-Type", "application/sparql-query")
    sparql_wrapper.setMethod(query_method)
    sparql_wrapper.setReturnFormat(response_format)
    sparql_wrapper.setQuery(query)
    response_object = sparql_wrapper.query()
    if sparql_wrapper.returnFormat == JSON:
        return response_object._convertJSON()
    return response_object.convert()
    

In [ ]:
# Create SPARQL query

query_example = """
SELECT ?s ?p ?o
WHERE {?s ?p ?o}
LIMIT 2
"""

In [ ]:
# Query sparql endpoint

results = query_sparql(sparql_endpoint, query_example)

In [ ]:
# Convert sparql json results into pandas DataFrame

def results_to_dataframe(json_results):
    columns = json_results["head"]["vars"]
    data = []
    for binding in json_results["results"]["bindings"]:
        row_values = []
        for col in columns:
            value = binding.get(col, {})["value"]
            row_values.append(value)
        data.append(row_values)
            
    return pd.DataFrame(data, columns=columns)

print(results_to_dataframe(results))

In [ ]:
# Save DataFrame to csv

def dataframe_to_csv(filename="sparql_results"):
    return results_to_dataframe(results).to_csv(f"{filename}.csv")

dataframe_to_csv()